In [9]:
from datetime import datetime
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from sqlalchemy import create_engine
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from src.scrape.scrape_threads import scrape_threads_range, scrape_top_threads, analyze_thread
from src.scrape.get_source_page import get_source_selenium
from src.load.load_data_to_postgresdb import load_user_staging_to_production, load_user_to_staging, load_thread_staging_to_production, load_thread_to_staging
from src.utilities.config import config
from src.utilities.database_utils import connect


Define params to scrape top threads from f

In [ ]:
top = 20
f = 33
last = 7
ext=f'?last_days={last}&order=reply_count&direction=desc'

In [11]:
top_n = scrape_top_threads(last_days=last,top=top,f=f, direction="desc")


KeyboardInterrupt



In [ ]:
# top_n = extract_threads_range(from_page=26, to_page=24, f=3, ext=ext)

In [ ]:
top_n

Remove duplicated threads

In [ ]:
r = set()
for i in top_n:
    if i['thread_id'] in r:
        top_n.remove(i)
    else:
        r.add(i['thread_id'])

This code block used only in case **scrapeing comments**. It will remove duplicated threads before scraping comments to
ensure we don't scrape comments of the same threads since this task takes long time

In [ ]:
engine = connect("postgresql_aws")
seen_id = engine.execute("SELECT thread_id from threads").fetchall()
seen_id_df = pd.DataFrame(seen_id)
not_seen = [i for i in top_n if i['thread_id'] not in seen_id_df.values]

In [ ]:
not_seen

No more covid threads!

In [ ]:
ignored_covid = [i for i in top_n if ("covid" not in i['title'].lower()) and ("corona" not in i['title'].lower()) and ("covid-19" not in (i['title'].lower()))]

Declare variables to store data

In [ ]:
host = 'https://voz.vn'
data_by_thread = {}
user = []

Start scraping comments

In [ ]:
for thread in not_seen:
    link = f'/t/{thread["thread_id"]}/'
    title = thread['title']
    page = 1
    data = []
    while True:
        url = f'{host}{link}page-{page}'
        driver = get_source_selenium(url)
        try:
            thread_stat, user_stat = analyze_thread(driver)
            data.extend(thread_stat)
            user.extend(user_stat)
        except:
            pass
        try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.LINK_TEXT, "Next"))
            )
            page += 1
            # driver.quit()
        except: # Reach last page
            print('Next not found')
            break
        finally:
            driver.quit()
    data_by_thread[title] = data

Remove `NULL` value in case failed to scrape users somehow

In [ ]:
res = [i for i in user if i is not None]

In [ ]:
df = pd.DataFrame(res)
df.drop_duplicates(subset="user_id", keep="last", inplace=True, ignore_index=True)
df['updated_at'] = datetime.now().strftime("%b %d, %Y %I:%M %p")
df['location'].replace("\\./", "", inplace=True) # Remove Escape keyword in location

Start loading users to databse

In [ ]:
engine = connect("postgresql_aws")

In [ ]:
load_user_to_staging(engine, "users_staging", df)

In [ ]:
load_user_staging_to_production(engine, staging_table="users_staging", prod_table="users")

This code block used only in case we scrape comments

In [ ]:
{i.update({'participants':data_by_thread[i['title']], 'updated_at':datetime.now().strftime("%b %d, %Y %I:%M %p"), 'f': f}) for i in not_seen[:10]}

This code block used only in case we **`don't`** scrape comments

In [ ]:
{i.update({'updated_at':datetime.now().strftime("%b %d, %Y %I:%M %p"), 'f': f}) for i in top_n[10:]}

Remember to modify the threads name properly

In [ ]:
load_thread_to_staging(engine, "threads_staging", not_seen)

In [ ]:
load_thread_staging_to_production(engine, staging_table="threads_staging", prod_table="threads")